In [1]:
!pip install slate3k

In [1]:
import re
import pandas as pd
import numpy as np
import slate3k as slate
import numpy as np
import pandas as pd
import glob
import warnings
import logging
import re
logging.getLogger().setLevel(logging.ERROR)
#pd.set_option('display.max_colwidth', -1)

In [22]:
DATA=pd.DataFrame(columns=['Contact','Top Skills','Languages', \
          'Certifications','Honors-Awards','Publications','Summary','Experience','Education','Designation_1','ExpDurationInMonths_1','Experience_Summary_1','Designation_2','ExpDurationInMonths_2','Experience_Summary_2','Education_1','EduDurationInMonths_1','Education_2','EduDurationInMonths_2','URL'])

In [23]:
DATA

,Contact,Top Skills,Languages,Certifications,Honors-Awards,Publications,Summary,Experience,Education,Designation_1,ExpDurationInMonths_1,Experience_Summary_1,Designation_2,ExpDurationInMonths_2,Experience_Summary_2,Education_1,EduDurationInMonths_1,Education_2,EduDurationInMonths_2,URL


In [15]:
keywords=['Contact','Top Skills','Languages', \
          'Certifications','Honors-Awards','Publications','Summary','Experience','Education']

In [16]:
def Exp_convert_months(x):
    start=x.index('(')
    end=x.index(')')
    y=x[start+1:end]
    z=list(y.split(' '))
    n1,n2=0,0
    n1 = [int(z[z.index(s)-1])*12 for s in z if any(xs in s for xs in ['year'])]
    n2=[int(z[z.index(s)-1]) for s in z if any(xs in s for xs in ['month'])]
    return sum(n1+n2)

In [17]:
def Edu_convert_months(x):
    if len(x)>1:
        a=int(x[0])
        b=int(x[1])
    else:
        a=int(x[0])
        b=int(2020)
    diffinmonths=(b-a)*12
    return diffinmonths

In [24]:
pdf_dir = "D:/LINKEDIN_DATA/SHILA/CVs_scrapped_linkedln_profile/CVs_scrapped_linkedln_profile/"
pdf_files = glob.glob("%s/*.pdf" % pdf_dir)
for file in pdf_files:
    with open(file,'rb') as f:
        extracted_text = slate.PDF(f)
        lst=list(extracted_text)
        listToStr = ' '.join(map(str, lst))
        listToStr=listToStr.replace('\xa0\n\xa0\n\xa0\n\x0c ','').replace('\xa0\n\xa0\n\xa0\n\x0c','')
    LIST=re.split('\n\n',listToStr)
    y,index=[],[]
    for i in range(len(keywords)):
        if keywords[i] in LIST:
            a=LIST.index(keywords[i])
            index.append(a)
            y.append(LIST[a])
        else:
             continue
    DICTIONARY={}
    i=0
    while i < len(index):
        if i==len(index)-1:
            DICTIONARY[LIST[index[i]]]=LIST[index[i]+1:]
        else:  
            DICTIONARY[LIST[index[i]]]=LIST[index[i]+1:index[i+1]]
        i=i+1
    try:
        Temp=DICTIONARY['Contact']
    except:
        continue
    j=0
    while j < len(Temp):
        if 'linkedin' in Temp[j]:
            if '\n' in Temp[j]:
                L=Temp[j].split('(')
                URL=L[0].replace('\n','')
            else:
                URL=Temp[j]
        j+=1
    DICTIONARY['URL']=URL
    if DICTIONARY.get("Experience")!=None:
            Experience=[]
            Designation=[]
            for i in DICTIONARY['Experience']:
                j=i.replace('\xa0','')
                if(re.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}-\bPresent\b[(]|(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}-Present|(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}-(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}[(]|\d{4}-\d{4}[(]',j)):
                    Designation.append(DICTIONARY['Experience'][DICTIONARY['Experience'].index(i)-1])
                    Experience.append(i)
                    if len(Experience)==2:
                        break
            summ_index,exp_summ=[],[]
            for i in Experience:
                if DICTIONARY['Experience'].index(i):
                    summ_index.append(DICTIONARY['Experience'].index(i))
            j=0
            l=len(summ_index)
            if j==(l-1):
                 exp_summ.append(DICTIONARY['Experience'][summ_index[j]+1:])
            else:
                while j < l:
                    if j==(l-1):
                        lst=DICTIONARY['Experience'][summ_index[j]+1:]
                        if len(lst)>0:
                            Tc=[]
                            for i in lst:
                                p=i.replace('\xa0','')
                                if(re.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}-\bPresent\b[(]|(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}-(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}[(]|\d{4}-\d{4}[(]',p)):
                                    Tc.append(DICTIONARY['Experience'].index(i))
                            if len(Tc)>0:
                                index=Tc[0]
                                exp_summ.append(DICTIONARY['Experience'][summ_index[j]+1:index-1])
                            else:
                                break
                        else:
                            break
                    else:
                        exp_summ.append(DICTIONARY['Experience'][summ_index[j]+1:summ_index[j+1]-1])
                    j+=1
            countDesig=1
            for a in Designation:
                DICTIONARY['Designation_'+str(countDesig)]=a
                countDesig+=1
            countExp=1
            for b in Experience:
                DICTIONARY['ExpDurationInMonths_'+str(countExp)]=int(Exp_convert_months(b))
                countExp+=1
            countSumm=1
            for c in exp_summ:
                DICTIONARY['Experience_Summary_'+str(countSumm)]=c
                countSumm+=1       
    if DICTIONARY.get("Education")!=None:
            Edu=[]
            Degree=[]
            Duration=[]
            for i in DICTIONARY['Education']:
                if(re.findall(r'\d{4}',i)):
                    Degree.append(i.replace('\xa0',''))
                Edu.append(re.findall(r'\d{4}',i))
                if len(Degree)==2:
                    break
            A=list(filter(None,Edu))
            for i in A:
                  Duration.append(Edu_convert_months(i)) 
            countDe=1
            for a in Degree:
                DICTIONARY['Education_'+str(countDe)]=a
                countDe+=1
            countDu=1
            for b in Duration:
                DICTIONARY['EduDurationInMonths_'+str(countDu)]=b
                countDu+=1
    DATA=DATA.append(DICTIONARY,ignore_index=True)

In [25]:
DATA.shape

(374, 20)

In [26]:
DATA

,Contact,Top Skills,Languages,Certifications,Honors-Awards,Publications,Summary,Experience,Education,Designation_1,ExpDurationInMonths_1,Experience_Summary_1,Designation_2,ExpDurationInMonths_2,Experience_Summary_2,Education_1,EduDurationInMonths_1,Education_2,EduDurationInMonths_2,URL
0,[www.linkedin.com/in/mayank-\njalan-1b05048 (L...,"[Business Development, Brand Management, Strat...",NaN,NaN,NaN,NaN,NaN,"[Indian Chamber of Commerce, Kolkata, Presiden...","[Harvard University, Agri Business Seminar · ...",President,9,[Keventer Agro],Managing Director,192,NaN,Agri Business Seminar·(2018-2018),0,"Bachelor of Engineering - BE,Mechanical Engine...",36,www.linkedin.com/in/mayank-jalan-1b05048
1,[www.linkedin.com/in/vbajaj\n(LinkedIn)\nwww.k...,"[Entrepreneurship, Seed Capital, Trading]",[],NaN,NaN,NaN,"[Top Skills, Entrepreneurship, Seed Capital, T...","[StockEdge, Co-Founder, May 2015 - Present (4 ...","[Indian Institute of Management, Indore, Maste...",Co-Founder,59,"[Empowering the individual retail investors, p...",Co-Founder,63,"[Co founder of elearnmarkets.com, a dedicated ...","Master of Business Administration (MBA),Financ...",24,"M.Com (H),Accounting and Finance·(1999-2001)",24,www.linkedin.com/in/vbajaj
2,[www.linkedin.com/in/vinod-\nsood-2131012 (Lin...,"[Service Delivery, Vendor Management, Telecomm...",NaN,NaN,NaN,NaN,NaN,"[Hughes Systique Corporation, Managing Directo...","[Punjab Engineering College, Bachelor's of Eng...",Managing Director,174,[Responsible for managing Hughes Systique Indi...,Member of the Board,31,[K.R. Mangalam University],Engineering·(1980-1984),48,Class XII·(1978-1980),24,www.linkedin.com/in/vinod-sood-2131012
3,"[www.linkedin.com/in/yashmahadik\n(LinkedIn), ...","[Talent Management, Leadership Development, Ta...","[Marathi, English, Gujrati]",NaN,[Duke of Edinburg's Award],"[What I learnt about Learning, Interview with ...",[Yashwant Mahadik (Yash) a seasoned global HR ...,"[Lupin Global, President - Global Human Resour...","[The University of Sheffield, Master of Busine...",President - Global Human Resources & Co-Lead B...,21,[Working with Lupin Limited as their Global He...,Executive Vice President & Global CHRO,32,"[The tour of duty was to help the company, its...",International Human Resources Management·(2002...,12,Accounting·(1985-1988),36,www.linkedin.com/in/yashmahadik
4,[www.linkedin.com/in/rohit-\ngawande27 (Linked...,"[Data Analytics, Business Analyst, Data Science]",NaN,[Data Science using Python and R],NaN,NaN,[RESUME],"[Capgemini, 1 year 1 month, Machine Learning E...","[Mumbai University Mumbai, Master's degree, Da...",Machine Learning Engineer,8,[],SAP Basis Consultant,13,[Techdata Solutions - India],"Master's degree,Data science·(2018-2020)",24,Engineering·(2013-2017),48,www.linkedin.com/in/rohit-gawande27
5,[www.linkedin.com/in/artikay\n(LinkedIn)\ney.c...,"[Business Development, Start-ups, Business Str...","[English, Hindi, Kannada (Elementary)]",NaN,NaN,[],"[Top Skills, Business Development, Start-ups, ...","[EY, Director, June 2018 - Present (1 year 10 ...","[Indian Institute of Management, Kozhikode, PG...",Director,22,"[I head the team that supports large, transfor...",Entrepreneur In Residence,34,[India's first on demand commercial bike taxi ...,"PGP,Finance, Operations, Marketing·(2006-2008)",24,"B.Tech.,Textile Technology·(1994-1999)",60,www.linkedin.com/in/artikay
6,[www.linkedin.com/in/amitabh-\nchattopadhyay-4...,"[Channel Management, Team Management, New Busi...",NaN,NaN,NaN,NaN,[A senior professional holding P&L accountabil...,"[Naukri.com, 8 years 7 months, Senior Vice Pre...","[Indian School of Business, · (2016 - 2016), ...",Senior Vice President,24,[],Vice President -Sales,77,"[Role:, - Acting as team head and managing the...",·(2016-2016),0,"MBA,Marketing , HR·(1993-1996)",36,www.linkedin.com/in/amitabh-chattopadhyay-4b23...
7,[www.linkedin.com/in/bhakti-jhaveri-\ndhanak-p...,"[Screening Resumes, Training Needs Analysis, E...",NaN,"[Digital Business Skills,

In [27]:
DATA.to_excel('D:/LINKEDIN_DATA/Excel/ShiladithData_March182020.xlsx')